# comercio_ext_auxiliares.tb_referencia_pais_bloco
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_BLOCO_delta`
## 📌 Descrição do arquivo
Referência de **Blocos Econômicos por País**.
|Coluna|Descrição|
|---|---|
|`CO_PAIS`|Código do país|
|`CO_BLOCO`|Código do bloco|
|`NO_BLOCO`|Nome do bloco (PT)|
|`NO_BLOCO_ING`|Nome do bloco (EN)|
|`NO_BLOCO_ESP`|Nome do bloco (ES)|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_BLOCO_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_referencia_pais_bloco/"   
val silverTable =    "tb_referencia_pais_bloco"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_PAIS" , StringType ,  nullable = false ) ,
      StructField ( "CO_BLOCO" , StringType ,  nullable = false ) ,
      StructField ( "NO_BLOCO" , StringType ,  nullable = false ) ,
      StructField ( "NO_BLOCO_ING" , StringType ,  nullable = false ) ,
      StructField ( "NO_BLOCO_ESP" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_PAIS", trim(upper(col("CO_PAIS").cast(StringType))))
  .withColumn("CO_BLOCO", trim(upper(col("CO_BLOCO").cast(StringType))))
  .withColumn("NO_BLOCO", col("NO_BLOCO").cast(StringType))
  .withColumn("NO_BLOCO_ING", col("NO_BLOCO_ING").cast(StringType))
  .withColumn("NO_BLOCO_ESP", col("NO_BLOCO_ESP").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/PAIS_BLOCO_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_BLOCO").isNotNull && col("CO_PAIS").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_BLOCO",
        "CO_PAIS"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_PAIS"), col("CO_BLOCO"), col("NO_BLOCO"), col("NO_BLOCO_ING"), col("NO_BLOCO_ESP"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}